### Data Ingestion

In [ ]:
###Document structure
from langchain_core.documents import Document
from pydantic_core.core_schema import none_schema
from pymupdf.mupdf import pdf_document

In [ ]:
doc = Document(
    page_content="This is the main content I'm using to create RAG",
    metadata={
        "source":"example.txt",
        "pages": 1,
        "author": "hai tran",
        "date_created": "17-12-2025"
    }
)
doc

In [ ]:
## Create a simple text file
import os
os.makedirs("../data/text_files", exist_ok=True)

In [ ]:
sample_text = {
    "../data/text_files/python_intro.txt": """
Python is a high-level, general-purpose programming language created by Guido van Rossum
and first released in 1991.

Python is known for its simple and readable syntax, which makes it a popular choice
for beginners as well as experienced developers.

The language supports multiple programming paradigms, including procedural,
object-oriented, and functional programming. Python is widely used in web development,
data science, artificial intelligence, automation, and scientific computing.
""",
    "../data/text_files/machine_learning.txt": """
Machine learning is a field of computer science that focuses on building systems
that can learn from data and improve their performance over time without being
explicitly programmed.

It is a core subfield of artificial intelligence and is commonly used in applications
such as recommendation systems, image recognition, speech recognition, and fraud detection.

Machine learning algorithms are typically categorized into supervised learning,
unsupervised learning, and reinforcement learning.
"""
}


for filepath, content in sample_text.items(): # items trả về key-value
    with open(filepath, "w") as f: # with cung cấp các helper method tương đưuong với việc gọi hàm __enter__ và __exit__ giúp đơn giản hóa
        f.write(content)

print("Sample text files created successfully!")

In [ ]:
### Textloader
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/text_files/python_intro.txt", encoding="utf-8")
document = loader.load()
print(document)


In [ ]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## Load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt", #Pattern to match files
    loader_cls=TextLoader, #Loader class to use for each file
    loader_kwargs={"encoding": "utf-8"},
    show_progress=False
)

document = dir_loader.load()
document

In [ ]:
### Directory Loader
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

## Load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/pdfs",
    glob="**/*.pdf", #Pattern to match files
    loader_cls=PyMuPDFLoader, #Loader class to use for each file
    show_progress=False
)

pdf_document = dir_loader.load()
pdf_document